<div>
<img src="https://www.ul.ie/themes/custom/ul/logo.jpg" />
</div>

#**MSc in Artificial Intelligence and Machine Learning**
##CS6271 - Evolutionary Algorithms and Humanoid Robotics 2025
### Kaggle Competition


Module Leader: Conor Ryan

Developer: Dhiraj Kumar Singh

Link to access the competition:

Link to join the competition:

Predict whether a person will earn more than 50k or less. This is a modified version of the adult dataset

In [2]:
!pip install deap

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from deap import base, creator, gp, tools, algorithms
import operator, math, random


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 3.4 MB/s eta 0:00:00


## Dataset

**Class:**

output: 1, 0.


**Listing of features:**

- 'age'
- 'workclass'
- 'fnlwgt'
- 'education'
- 'education-num'
- 'marital-status'
- 'occupation'
- 'relationship'
- 'race'
- 'sex'
- 'capital-gain'
- 'capital-loss'
- 'hours-per-week'
- 'native-country'

In [3]:
# Suppressing Warnings:
import warnings
warnings.filterwarnings("ignore")

## Train set

In [4]:

train_file = '/content/train.csv'
df_train = pd.read_csv(train_file)
df_train.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,37,Private,193106,Bachelors,13,Never-married,Sales,Not-in-family,White,Female,0,0,30,United-States,0
1,56,Self-emp-inc,216636,12th,8,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1651,40,United-States,0
2,53,Private,126977,HS-grad,9,Separated,Craft-repair,Not-in-family,White,Male,0,0,35,United-States,0
3,72,Private,205343,11th,7,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0
4,46,State-gov,106705,Masters,14,Never-married,Exec-managerial,Not-in-family,White,Female,0,0,38,United-States,0


In [5]:
df_train.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,income
count,39073.000000,3.907300e+04,39073.000000,39073.000000,39073.000000,39073.000000,39073.000000
mean,38.643488,1.899922e+05,10.069844,1038.040540,86.807949,40.476877,0.239270
std,13.685634,1.054768e+05,2.574387,7204.953114,401.276773,12.401251,0.426643
min,17.000000,1.376900e+04,1.000000,0.000000,0.000000,1.000000,0.000000
25%,28.000000,1.177670e+05,9.000000,0.000000,0.000000,40.000000,0.000000
50%,37.000000,1.786150e+05,10.000000,0.000000,0.000000,40.000000,0.000000
75%,48.000000,2.383290e+05,12.000000,0.000000,0.000000,45.000000,0.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,4356.000000,99.000000,1.000000


In [6]:
df_train.describe(include='object')

,workclass,education,marital-status,occupation,relationship,race,sex,native-country
count,38304,39073,39073,38302,39073,39073,39073,38851
unique,9,16,7,15,6,5,2,42
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States
freq,27141,12582,17951,4915,15822,33414,26170,35059


In [7]:
train_X = df_train.copy()
# warning: cannot drop it more than once
train_X.drop(['income'], axis=1, inplace=True)

train_y = df_train['income'].to_numpy()

## Test set: **No labels**

In [8]:

train_file = 'test.csv'
df_test = pd.read_csv(train_file)
df_test.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,30,Private,378009,HS-grad,9,Never-married,Machine-op-inspct,Own-child,White,Male,0,0,40,United-States
1,54,State-gov,55861,Assoc-acdm,12,Divorced,Adm-clerical,Not-in-family,White,Male,0,0,39,United-States
2,21,?,204226,Some-college,10,Never-married,?,Unmarried,White,Female,0,0,35,United-States
3,35,Private,306678,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,2885,0,40,United-States
4,42,Local-gov,121012,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States


In [9]:
df_test.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,9769.000000,9.769000e+03,9769.000000,9769.000000,9769.000000,9769.000000
mean,38.643976,1.883518e+05,10.111066,1243.163374,90.279558,40.204422
std,13.810263,1.061065e+05,2.557138,8365.976609,409.851357,12.350369
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,27.000000,1.164930e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.767320e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.345420e+05,13.000000,0.000000,0.000000,45.000000
max,90.000000,1.366120e+06,16.000000,99999.000000,3770.000000,99.000000


In [10]:
df_test.describe(include='object')

,workclass,education,marital-status,occupation,relationship,race,sex,native-country
count,9575,9769,9769,9574,9769,9769,9769,9717
unique,9,16,7,15,6,5,2,41
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States
freq,6765,3202,4428,1257,3894,8348,6480,8773


In [11]:
test_X = df_test.copy()

**Data Preprocessing**

Replaces missing values (‘?’) with NaN and fills them using the mode of each column in both training and test sets.

In [12]:
train_X = train_X.replace('?', np.nan)
test_X = test_X.replace('?', np.nan)
train_X = train_X.fillna(train_X.mode().iloc[0])
test_X = test_X.fillna(test_X.mode().iloc[0])

Preprocesses train and test sets together for consistent encoding. Categorical features are label-encoded, numerical features scaled. Training data is split 80/20 for validation; test data is scaled separately for final predictions.

In [13]:
combined = pd.concat([train_X, test_X], axis=0, ignore_index=True)

for col in combined.columns:
    if combined[col].dtype == 'object':
        le = LabelEncoder()
        combined[col] = le.fit_transform(combined[col].astype(str))

train_X = combined.iloc[:len(df_train), :].copy()
test_X  = combined.iloc[len(df_train):, :].copy()

scaler = StandardScaler()
train_X_scaled = scaler.fit_transform(train_X)
test_X_scaled  = scaler.transform(test_X)


X_train, X_val, y_train, y_val = train_test_split(train_X_scaled, train_y, test_size=0.2, random_state=42)

## Genetic Programming

This section defines the GP function set, including safe mathematical operations, nonlinear functions, and random constants. These primitives allow the GP algorithm to build stable and expressive symbolic models.

In [14]:
def safeDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

def safeLog(x):
    try:
        return math.log(abs(x)+1)
    except ValueError:
        return 0

pset = gp.PrimitiveSet("MAIN", X_train.shape[1])
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(safeDiv, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(safeLog, 1)
pset.addPrimitive(math.sin, 1)
pset.addPrimitive(math.cos, 1)
pset.addEphemeralConstant("rand101", lambda: random.uniform(-1, 1))
pset.renameArguments(**{f"ARG{i}": f"x{i}" for i in range(X_train.shape[1])})


This section defines the GP hyperparameters, including population size, number of generations, crossover/mutation probabilities, and tree-depth limits. These settings control how the GP algorithm evolves models over time.

In [15]:
POPULATION_SIZE = 800
MAX_GENERATIONS = 100
P_CROSSOVER = 0.9
P_MUTATION = 0.4
HALL_OF_FAME_SIZE = 15

MIN_TREE_HEIGHT = 1
MAX_TREE_HEIGHT = 7
LIMIT_TREE_HEIGHT = 35
MUT_MIN_TREE_HEIGHT = 0
MUT_MAX_TREE_HEIGHT = 3
RANDOM_SEED = 42
random.seed(RANDOM_SEED)


This function compiles a GP tree into an executable function, applies it to every training sample, converts outputs into binary predictions, and computes accuracy. The accuracy score becomes the fitness value used for evolution.

In [16]:
def eval_individual(individual):
    func = toolbox.compile(expr=individual)
    preds = []
    for row in X_train:
        try:
            val = func(*row)
        except Exception:
            val = 0
        preds.append(1 if val > 0 else 0)
    acc = accuracy_score(y_train, preds)
    return (acc,)

Sets up the GP framework: defines the fitness, individual representation, population, genetic operators, and evaluation method. Also limits tree sizes to prevent overgrowth.

In [17]:
toolbox = base.Toolbox()
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=MIN_TREE_HEIGHT, max_=MAX_TREE_HEIGHT)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("evaluate", eval_individual)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=MUT_MIN_TREE_HEIGHT, max_=MUT_MAX_TREE_HEIGHT)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)
toolbox.decorate("mate", gp.staticLimit(key=len, max_value=LIMIT_TREE_HEIGHT))
toolbox.decorate("mutate", gp.staticLimit(key=len, max_value=LIMIT_TREE_HEIGHT))


Runs the GP evolutionary loop: initializes the population, tracks the best individuals, collects statistics, and evolves solutions over generations. The best individual is compiled into a callable function.

In [18]:
random.seed(RANDOM_SEED)
pop = toolbox.population(n=POPULATION_SIZE)
hof = tools.HallOfFame(HALL_OF_FAME_SIZE)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("max", np.max)

pop, log = algorithms.eaSimple(pop, toolbox,
                               cxpb=P_CROSSOVER, mutpb=P_MUTATION,
                               ngen=MAX_GENERATIONS, stats=stats,
                               halloffame=hof, verbose=True)

print("\nBest individual:", hof[0])
best_func = toolbox.compile(expr=hof[0])

gen	nevals	avg     	max     
0  	800   	0.446524	0.799411
1  	747   	0.5228  	0.799411
2  	738   	0.556116	0.798835
3  	739   	0.598321	0.785303
4  	756   	0.625225	0.794357
5  	747   	0.653893	0.78911 
6  	744   	0.681956	0.799795
7  	756   	0.68059 	0.800467
8  	740   	0.697106	0.818255
9  	740   	0.697595	0.830251
10 	746   	0.700147	0.830219
11 	755   	0.700848	0.830219
12 	762   	0.711255	0.830219
13 	750   	0.717957	0.830219
14 	745   	0.725164	0.830475
15 	747   	0.722446	0.829516
16 	768   	0.719493	0.820206
17 	756   	0.718025	0.820206
18 	758   	0.722773	0.819982
19 	760   	0.723478	0.820398
20 	751   	0.728659	0.820398
21 	762   	0.725635	0.839465
22 	754   	0.733469	0.839465
23 	763   	0.736897	0.839465
24 	749   	0.741059	0.839465
25 	742   	0.742034	0.838985
26 	754   	0.748061	0.839433
27 	754   	0.737783	0.839913
28 	752   	0.74175 	0.839913
29 	745   	0.754741	0.839913
30 	754   	0.746133	0.838921
31 	752   	0.747248	0.839113
32 	746   	0.757982	0.839113
33 	756   	0.7

Calculating Accuracies - Training & Validation



In [22]:
# Training accuracy predictions
train_preds = [1 if best_func(*x) > 0 else 0 for x in X_train]
train_acc = accuracy_score(y_train, train_preds)
print("Training Accuracy:", train_acc)


Training Accuracy: 0.8427282615650393


In [25]:
#Validation accuracy prediction
val_preds = [1 if best_func(*x) > 0 else 0 for x in X_val]
val_acc = accuracy_score(y_val, val_preds)
print("Validation Accuracy:", val_acc)


Validation Accuracy: 0.8349328214971209


Calculating the test accuracy and creating the submission csv file

In [24]:
test_preds = [1 if best_func(*x) > 0 else 0 for x in test_X_scaled]

submission = pd.DataFrame({
    "id": np.arange(len(test_preds)),
    "income": test_preds
})

submission.to_csv("submission.csv", index=False)
print("Submission file saved as submission.csv")


Submission file saved as submission.csv
